In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%load_ext rpy2.ipython

In [3]:
import os
import numpy as np
import pandas as pd
import dual_crispr_screen
import imp
imp.reload(dual_crispr_screen)



<module 'dual_crispr_screen' from '/cellar/users/samsonfong/crappy/src/dual_crispr_screen.py'>

In [4]:
# Global parameters

BASE = os.path.realpath('../')
TEST_SET = os.path.join(BASE, 'data', 'test_data','input_data', 'TestSet8')

# Test Set

In [12]:
abundance_file = os.path.join(TEST_SET, "TestSet8_abundance_thresholds.txt")
counts_file = os.path.join(TEST_SET, "TestSet8_timepoint_counts.txt")

#abundance_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/TestSet8/TestSet8_abundance_thresholds.txt"
#counts_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/TestSet8/TestSet8_timepoint_counts.txt"
times = np.array([21,28])

# initialize screen object
screen = dual_crispr_screen.Screen(counts_file, times, abundance_file)

# set some parameters
# number of iterations to perform
screen.options.niter = 2
screen.options.testing = True
screen.options.null_target_id = '0'
screen.options.verbose = True

# perform construct fitness calculation
screen.run_construct_fitting()

# run bootstraps
screen.run_bootstrap()

# summarize results 
screen.summarize()

# view results 
screen.results.head()

hi
               log2CountsThresh
sampleName                     
A549CV4_T21_1             4.525
A549CV4_T21_2             4.325
A549CV4_T28_1             3.925
A549CV4_T28_2             3.475
[[[-18.4583628  -19.32780328]]

 [[-18.66790896 -19.97675167]]]


/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:257: RuntimeWarning: invalid value encountered in true_divide
  
/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:283: RuntimeWarning: invalid value encountered in true_divide
  


Iter	RMS	Relative Error
1	0.0227	0.058415
2	0.0219	0.030430
3	0.0215	0.016396
4	0.0213	0.009132
5	0.0211	0.005257
6	0.0211	0.003128
7	0.0210	0.001920
8	0.0210	0.001211
9	0.0210	0.000783
Performing iteration: 1
Iter	RMS	Relative Error
1	0.0014	0.084903
2	0.0010	0.008124
3	0.0009	0.000827
Performing iteration: 2
Iter	RMS	Relative Error
1	0.0013	0.083478
2	0.0010	0.008159
3	0.0009	0.000923


/cellar/users/samsonfong/crappy/src/bootstrap_pi.py:274: RuntimeWarning: divide by zero encountered in true_divide
  fdr_right = np.minimum(1, enull(-pi_mean)/(1-emean(pi_mean)))


,geneA,geneB,pi_mean,fdr_left,fdr_right,sd,z,pp,fA,fB
0,0,ABL1,-0.016598,0.520198,1.0,0.025642,-0.646978,0.0,-0.000108,0.000593
1,0,ADA,0.016067,0.936011,1.0,0.022820,0.626272,0.0,-0.000108,0.001239
2,0,AKT1,-0.014196,0.555413,1.0,0.023174,-0.553338,0.0,-0.000108,0.000690
3,0,ALK,-0.001825,0.699888,1.0,0.005062,-0.071121,0.0,-0.000108,0.000725
4,0,APC,-0.029941,0.432203,1.0,0.044673,-1.167065,0.0,-0.000108,0.000575


In [6]:
### Compare to CTG benchmark pi scores

In [7]:
#benchmark = "/cellar/users/bpmunson/crappy/data/test_data/output_data/Notebook8Test_pi.txt"
benchmark = os.path.join(BASE, 'data', 'test_data', 'output_data', 'Notebook8Test_pi.txt')

benchmark = pd.read_csv(benchmark, sep="\t", header=0)
benchmark_pi_mean = benchmark[['geneA','geneB','pi']]
benchmark_pi_mean = benchmark_pi_mean.sort_values(['geneA','geneB']).reset_index(drop=True)

#pi_scores_mean = screen.results.mean(axis=1).reset_index()
pi_scores_mean = screen.results[['geneA','geneB','pi_mean']]

comb = pd.merge(pi_scores_mean, benchmark_pi_mean, on=['geneA','geneB'])

print('All pi-scores close:')
np.allclose(comb['pi_mean'],comb['pi'], atol=1e-6, equal_nan=True)

All pi-scores close:


True

In [8]:
%%R -i comb
jointPlot <- function(df, alpha=0.1, title=NULL, correlation=FALSE){
    
    # make a joint plot from the data frame of two columns first one is on x axis
    require(ggplot2)
    require(cowplot)
    # get info
    x_l = colnames(df)[1]
    y_l = colnames(df)[2]
    coord_min = min(na.omit(df))
    coord_max = max(na.omit(df))
    # make scatter plot
    scatter = ggplot(df, aes_string(x=x_l,y=y_l))+ # init grob
        geom_point(shape=1, alpha=alpha, size=4)+ # add points
        geom_abline(slope=1,intercept=0,color="red",linetype="dotted")+ # add perfect 1:1 line
        theme_bw(base_size=18)+ # use preset theme
        xlim(coord_min,coord_max)+ylim(coord_min,coord_max)+ # make square plot
        labs(x=x_l,y=y_l) # add axis labels
    # add correlation info to scatter plot
    if (correlation){
        # get correaltion
        res = cor.test(df[,x_l], df[,y_l])
        p_value = res$p.value
        corr = res$estimate
        # add annotation
        scatter = scatter + annotate("text",
                                     hjust=0,
                                     label=paste("Corr: ", format(round(corr, 4), nsmall=3),
                                                 sep=""),
                                     x=coord_min,y=coord_max)
        scatter = scatter + geom_smooth(se=FALSE, method="lm")
    }

    # add title to scatter plot
    if (!is.null(title)){
        scatter = scatter + labs(title=title)
    }

    xdens = axis_canvas(scatter, axis="x")+
                geom_histogram(data=df, aes_string(x=x_l), color="black",fill="white",bins=50)

    ydens = axis_canvas(scatter, axis="y", coord_flip=TRUE)+
                geom_histogram(data=df, aes_string(x=y_l), color="black",fill="white",bins=50)+
                coord_flip()

    p1 <- insert_xaxis_grob(scatter, xdens, grid::unit(.2,'null'),position="top")
    p2 <- insert_yaxis_grob(p1, ydens, grid::unit(.2,'null'),position="right")
    ggdraw(p2)
    
}
df = comb[,c('pi_mean','pi')]
colnames(df) = c('CrAPPY', 'CTG')
jointPlot(df, title="Test Set 8 Pi Scores", correlation=TRUE)



Error in axis_canvas(scatter, axis = "x") : 
  could not find function "axis_canvas"


/cellar/users/samsonfong/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)
/cellar/users/samsonfong/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: cowplot

  warnings.warn(x, RRuntimeWarning)
/cellar/users/samsonfong/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in axis_canvas(scatter, axis = "x") : 
  could not find function "axis_canvas"

  warnings.warn(x, RRuntimeWarning)
/cellar/users/samsonfong/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/cellar/users/samsonfong/miniconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/cellar/users/samsonfong/miniconda3/lib/python3.5/site-package

# Testing abundance thresholds

In [13]:
abundance_file = os.path.join(TEST_SET, "TestSet8_abundance_thresholds.txt")
counts_file = os.path.join(TEST_SET, "TestSet8_timepoint_counts.txt")

times = np.array([21,28])

# initialize screen object
screen = dual_crispr_screen.Screen(counts_file, times)

# set some parameters
# number of iterations to perform
screen.options.niter = 2
screen.options.testing = True
screen.options.null_target_id = '0'
screen.options.verbose = True

# perform construct fitness calculation
screen.run_construct_fitting()

# run bootstraps
screen.run_bootstrap()

# summarize results 
screen.summarize()

# view results 
screen.results.head()

hi
here
               log2CountsThresh
sampleName                     
A549CV4_T21_1             4.525
A549CV4_T21_2             3.025
A549CV4_T28_1             3.025
A549CV4_T28_2             3.025
[[[-18.4583628  -20.22780328]]

 [[-19.96790896 -20.42675167]]]


/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:257: RuntimeWarning: invalid value encountered in true_divide
  
/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:283: RuntimeWarning: invalid value encountered in true_divide
  


Iter	RMS	Relative Error
1	0.0230	0.064847
2	0.0220	0.033160
3	0.0215	0.017580
4	0.0213	0.009654
5	0.0212	0.005502
6	0.0211	0.003252
7	0.0210	0.001987
8	0.0210	0.001248
9	0.0210	0.000803
Performing iteration: 1
Iter	RMS	Relative Error
1	0.0013	0.095154
2	0.0009	0.008894
3	0.0009	0.000900
Performing iteration: 2
Iter	RMS	Relative Error
1	0.0013	0.091189
2	0.0010	0.007978
3	0.0010	0.000828


/cellar/users/samsonfong/crappy/src/bootstrap_pi.py:274: RuntimeWarning: divide by zero encountered in true_divide
  fdr_right = np.minimum(1, enull(-pi_mean)/(1-emean(pi_mean)))


,geneA,geneB,pi_mean,fdr_left,fdr_right,sd,z,pp,fA,fB
0,0,ABL1,-0.017435,0.618598,1.0,0.024352,-0.591611,1.0,0.000035,0.000330
1,0,ADA,0.020173,0.926233,1.0,0.027724,0.684517,1.0,0.000035,0.000521
2,0,AKT1,-0.026411,0.549815,1.0,0.037728,-0.896172,0.0,0.000035,0.001011
3,0,ALK,-0.001688,0.764569,1.0,0.002553,-0.057265,0.0,0.000035,-0.000044
4,0,APC,-0.022500,0.590545,1.0,0.074923,-0.763473,0.0,0.000035,-0.000179


In [14]:
#benchmark = "/cellar/users/bpmunson/crappy/data/test_data/output_data/Notebook8Test_pi.txt"
benchmark = os.path.join(BASE, 'data', 'test_data', 'output_data', 'Notebook8Test_pi.txt')

benchmark = pd.read_csv(benchmark, sep="\t", header=0)
benchmark_pi_mean = benchmark[['geneA','geneB','pi']]
benchmark_pi_mean = benchmark_pi_mean.sort_values(['geneA','geneB']).reset_index(drop=True)

#pi_scores_mean = screen.results.mean(axis=1).reset_index()
pi_scores_mean = screen.results[['geneA','geneB','pi_mean']]

comb = pd.merge(pi_scores_mean, benchmark_pi_mean, on=['geneA','geneB'])

print('All pi-scores close:')
np.allclose(comb['pi_mean'],comb['pi'], atol=1e-6, equal_nan=True)

All pi-scores close:


False

In [17]:
def rmse(x,y): 
    return np.power(np.sum(np.power(x, 2) - np.power(y,2))/len(x), 0.5)

In [18]:
rmse(comb['pi_mean'], comb['pi'])

0.014795560938966215

In [20]:
np.corrcoef(comb[['pi_mean', 'pi']].T)

array([[ 1.        ,  0.84489852],
       [ 0.84489852,  1.        ]])

# A549

In [25]:
abundance_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/A549/A549_abundance_thresholds.txt"
counts_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/A549/A549_timepoint_counts.txt"
times = np.array([3,14,21,28])

# initialize screen object
screen = dual_crispr_screen.Screen(counts_file, times)

# set some parameters
# number of iterations to perform
screen.options.niter = 2
screen.options.null_target_id = '0'
screen.options.verbose = True

# perform construct fitness calculation
screen.run_construct_fitting()

# run bootstraps
screen.run_bootstrap()

# summarize results 
screen.summarize()

# view results 
screen.results.head()

hi
here
               log2CountsThresh
sampleName                     
testing_T3_1              3.025
testing_T3_2              3.525
testing_T14_1             5.525
testing_T14_2             3.025
testing_T21_1             4.525
testing_T21_2             3.025
testing_T28_1             3.025
testing_T28_2             3.525
[[[-19.94704005 -17.32743826 -18.45876672 -20.62162419]]

 [[-19.81480163 -20.0044399  -19.96829224 -20.36940342]]]


/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:257: RuntimeWarning: invalid value encountered in true_divide
  fc = np.divide(f.sum(axis=0), var_time.sum(axis=0))
/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:283: RuntimeWarning: invalid value encountered in true_divide
  sdfc = np.divide(num, denom).data


Iter	RMS	Relative Error
1	0.0238	0.030505
2	0.0234	0.018467
3	0.0233	0.013551
4	0.0233	0.011265
5	0.0233	0.009165
6	0.0233	0.006738
7	0.0233	0.004585
8	0.0233	0.003149
9	0.0233	0.002354
10	0.0233	0.001940
11	0.0232	0.001711
12	0.0232	0.001570
13	0.0232	0.001472
14	0.0232	0.001396
15	0.0232	0.001329
16	0.0231	0.001269
17	0.0231	0.001211
18	0.0231	0.001154
19	0.0231	0.001096
20	0.0231	0.001036
21	0.0231	0.000974
Performing iteration: 1
Iter	RMS	Relative Error
1	0.0172	0.072186
2	0.0151	0.038327
3	0.0140	0.021314
4	0.0134	0.012327
5	0.0131	0.007436
6	0.0129	0.004669
7	0.0128	0.003025
8	0.0127	0.002010
9	0.0127	0.001361
10	0.0126	0.000934
Performing iteration: 2
Iter	RMS	Relative Error
1	0.0177	0.071938
2	0.0156	0.039396
3	0.0143	0.024304
4	0.0136	0.016121
5	0.0132	0.010819
6	0.0129	0.007122
7	0.0128	0.004608
8	0.0127	0.002954
9	0.0127	0.001896
10	0.0127	0.001224
11	0.0127	0.000798


/cellar/users/samsonfong/crappy/src/bootstrap_pi.py:274: RuntimeWarning: invalid value encountered in true_divide
  fdr_right = np.minimum(1, enull(-pi_mean)/(1-emean(pi_mean)))


,geneA,geneB,pi_mean,fdr_left,fdr_right,sd,z,pp,fA,fB
0,0,ABL1,0.012204,0.929617,1.0,0.009736,0.309485,1.0,-0.000434,-0.001625
1,0,ADA,0.006630,0.870163,1.0,0.006309,0.168129,1.0,-0.000434,-0.010265
2,0,AKT1,0.013931,0.946090,1.0,0.012539,0.353268,1.0,-0.000434,-0.019914
3,0,ALK,-0.086508,0.027304,1.0,0.003064,-2.193763,1.0,-0.000434,-0.010346
4,0,APC,0.021007,0.982543,1.0,0.015873,0.532713,1.0,-0.000434,-0.002217


In [22]:
abundance_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/A549/A549_abundance_thresholds.txt"
counts_file = "/cellar/users/bpmunson/crappy/data/test_data/input_data/A549/A549_timepoint_counts.txt"
times = np.array([3,14,21,28])

# initialize screen object
screen = dual_crispr_screen.Screen(counts_file, times, abundance_file)

# set some parameters
# number of iterations to perform
screen.options.niter = 2
screen.options.null_target_id = '0'
screen.options.verbose = True

# perform construct fitness calculation
screen.run_construct_fitting()

# run bootstraps
screen.run_bootstrap()

# summarize results 
screen.summarize()

# view results 
screen.results.head()

hi
               log2CountsThresh
sampleName                     
testing_T3_1           3.335947
testing_T3_2           3.475000
testing_T14_1          3.338280
testing_T14_2          4.089135
testing_T21_1          4.525000
testing_T21_2          4.325000
testing_T28_1          4.225000
testing_T28_2          3.575000
[[[-19.63609302 -19.51415828 -18.45876672 -19.42162419]]

 [[-19.86480163 -18.94030516 -18.66829224 -20.31940342]]]


/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:257: RuntimeWarning: invalid value encountered in true_divide
  
/cellar/users/samsonfong/crappy/src/fit_ac_fc.py:283: RuntimeWarning: invalid value encountered in true_divide
  


Iter	RMS	Relative Error
1	0.0234	0.029609
2	0.0231	0.017972
3	0.0229	0.013193
4	0.0229	0.011195
5	0.0229	0.009443
6	0.0229	0.007252
7	0.0229	0.005264
8	0.0229	0.004032
9	0.0228	0.003460
10	0.0228	0.003224
11	0.0227	0.003122
12	0.0227	0.003071
13	0.0226	0.003037
14	0.0226	0.003005
15	0.0225	0.002964
16	0.0225	0.002900
17	0.0225	0.002795
18	0.0224	0.002640
19	0.0224	0.002437
20	0.0224	0.002191
21	0.0224	0.001913
22	0.0223	0.001629
23	0.0223	0.001359
24	0.0223	0.001117
25	0.0223	0.000907
Performing iteration: 1
Iter	RMS	Relative Error
1	0.0181	0.069817
2	0.0161	0.037251
3	0.0149	0.022218
4	0.0142	0.014551
5	0.0137	0.009962
6	0.0134	0.006875
7	0.0132	0.004726
8	0.0131	0.003237
9	0.0131	0.002217
10	0.0130	0.001520
11	0.0130	0.001044
12	0.0130	0.000719
Performing iteration: 2
Iter	RMS	Relative Error
1	0.0169	0.072222
2	0.0148	0.039524
3	0.0135	0.023752
4	0.0128	0.014835
5	0.0124	0.009270
6	0.0122	0.005768
7	0.0121	0.003597
8	0.0120	0.002263
9	0.0120	0.001444
10	0.0120	0.000938


/cellar/users/samsonfong/crappy/src/bootstrap_pi.py:274: RuntimeWarning: invalid value encountered in true_divide
  fdr_right = np.minimum(1, enull(-pi_mean)/(1-emean(pi_mean)))


,geneA,geneB,pi_mean,fdr_left,fdr_right,sd,z,pp,fA,fB
0,0,ABL1,-0.009842,0.563918,1.000000,0.112178,-0.258908,0.0,-0.001188,-0.004513
1,0,ADA,0.044219,1.000000,0.808081,0.021166,1.163243,1.0,-0.001188,-0.009226
2,0,AKT1,0.011950,0.937530,1.000000,0.008075,0.314351,1.0,-0.001188,-0.017353
3,0,ALK,-0.040308,0.154255,1.000000,0.003147,-1.060359,1.0,-0.001188,-0.009673
4,0,APC,-0.015656,0.431523,1.000000,0.010487,-0.411856,1.0,-0.001188,-0.001830


In [ ]:
sampleName                     
testing_T3_1              3.025
testing_T3_2              3.525
testing_T14_1             5.525
testing_T14_2             3.025
testing_T21_1             4.525
testing_T21_2             3.025
testing_T28_1             3.025
testing_T28_2             3.525



sampleName                     
testing_T3_1           3.335947
testing_T3_2           3.475000
testing_T14_1          3.338280
testing_T14_2          4.089135
testing_T21_1          4.525000
testing_T21_2          4.325000
testing_T28_1          4.225000
testing_T28_2          3.575000

In [23]:
2**3.025

8.13983753682149